In [3]:
from __future__ import print_function
import os
import gmaps
import googlemaps
import argparse
import numpy as np
import pandas as pd
import json
from ipywidgets.embed import embed_minimal_html
import time
import tsplib95
import networkx as nx
import scipy
import os
import subprocess
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from bs4 import BeautifulSoup
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
import uuid
import time


def get_random_filename(ext=None):
    tmp_filename = uuid.uuid4()
    return str(tmp_filename) + '.{}'.format(ext) if ext else str(tmp_filename)


def get_datetime():
    return time.time()


def get_random_filename_with_datetime(ext=None, sep='-'):
    tmp_filename = uuid.uuid4()
    filename = str(tmp_filename) + '{}{}'.format(sep, time.time()) + '.{}'.format(ext) if ext else ''
    return filename

def print_problem_summary(num_stores, depart_time, finish_time=None):
    print('Total Store: ', num_stores)
    print('Total Warehouse: 1')
    print('Departure time: ', depart_time)
    print('Finished time: ', finish_time)

class Problem:
    def __init__(self, prob_type, metadata):
        self.prob_type = prob_type
        self.metadata = metadata

class LKH:
    def __init__(self, settings=None):
        self.settings = settings

    def solve(self, problem):
        if problem.prob_type == 'tsptw':
            return self.find_tsptw_solution(problem.metadata)

    def find_tsptw_solution(self, metadata):
        id_filename = self.write_tsplib95_format(
                        matrix=metadata['matrix'], 
                        time_window=metadata['time_window'],
                        depot=metadata['depot'],
                        filename_id=metadata['filename_id'])
        par_filename = self.write_par_file(id_filename)
        return self.execute_cmd(par_filename, id_filename)

    @staticmethod
    def execute_cmd(par_filename, id_filename):
        result = subprocess.run(['../solver/LKH-3.0.6/LKH', '{}'.format(par_filename)],
                                stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#         for l in result.stdout.decode("utf-8") .split('\n'):
#             print(l)
#         for l in result.stderr.decode("utf-8") .split('\n'):
#             print(l)
        if os.path.exists('../format/{}.sol'.format(id_filename)):
            sol = tsplib95.load_solution('../format/{}.sol'.format(id_filename))
            return sol.tours[0]
        return None

    @staticmethod
    def write_par_file(id_filename):
        filename = '../format/{}.par'.format(id_filename)
        file = open(filename, 'w')
        file.write('PROBLEM_FILE = {}\n'.format('../format/{}.tsptw'.format(id_filename)))
        file.write('TOUR_FILE = {}\n'.format('../format/{}.sol'.format(id_filename)))
        file.close()
        return filename

    def write_tsplib95_format(self, **kwargs):
        id_filename = kwargs['filename_id']
        filename = '../format/{}.tsptw'.format(id_filename)
        file = open(filename, 'w')
        file.write('NAME : {}\n'.format('{}.tsptw'.format(id_filename)))
        file.write('TYPE : {}\n'.format('TSPTW'))
        file.write('DIMENSION : {}\n'.format(len(kwargs['matrix'])))
        file.write('EDGE_WEIGHT_TYPE : {}\n'.format('EXPLICIT'))
        file.write('EDGE_WEIGHT_FORMAT : {}\n'.format('FULL_MATRIX'))
        file.write('EDGE_WEIGHT_SECTION\n')
        for arr in kwargs['matrix']:
            file.write(' '.join([str(int(e)) for e in arr]) + '\n')
        file.write('TIME_WINDOW_SECTION\n')
        for i, arr in enumerate(kwargs['time_window']):
            file.write(str(i + 1) + ' ' + ' '.join([str(e) for e in arr]) + '\n')
        file.write('DEPOT_SECTION\n')
        file.write(str(kwargs.get('depot', 0) + 1) + '\n')
        file.write('-1\n')
        file.write('EOF')
        file.close()
        return id_filename

class TSPTW:
    def __init__(self, static_data=None, metric='duration'):
        self.static_data = static_data
        self.metric = metric
        if static_data is None or metric=='duration':
            self.gg_client = googlemaps.Client(key=os.getenv('GOOGLE_API_KEY'))

    def read(self, input_tsp):
        self._read_json_from_file(input_tsp)
        self.input_tsp = input_tsp

    def run(self):
        self.metadata = self.data['metadata']
        self.addresses = [descode['formatted_address'] for descode in self.problem['destination']]
        self.center_point = self.get_center_map(self.problem)
        self.convered_point = self.get_covered_point(self.problem)
        self.depot = [self.get_location(self.get_depot(self.problem))]
        self.destinations = [self.get_location(descode) for descode in self.problem['destination']]
        self.optimal_tsp = self.find_optimal_tsptw(self.problem['depot'], self.destinations, self.problem['time_window'])
        self.descriptions = self.get_direction_description(self.optimal_tsp)
    
    def find_optimal_tsptw(self, depot, destinations, time_window):
        logger.info('Gathering distance matrix and duration matrix')
        distance_matrix, duration_matrix = self.get_utility_matrices(destinations, destinations)
        metadata = {'matrix': duration_matrix if self.metric == 'duration' else distance_matrix, 
                    'time_window': time_window, 
                    'depot': depot, 
                    'filename_id': self.input_tsp[self.input_tsp.rfind('/') + 1: -5]}
        problem = Problem('tsptw', metadata)
        tool = LKH()
        solution = tool.solve(problem)
        return solution

    def get_location(self, geocode):
        return (geocode['geometry']['location']['lat'], geocode['geometry']['location']['lng'])

    def _read_json_from_file(self, json_loc):
        logger.info('Reading data from JSON file')
        self.input_tsp = json_loc
        self.data = json.load(open(json_loc, 'r'))
        self.problem = self.data['problem']

    def get_depot(self, data):
        logger.info('Read depot location')
        depot_idx = data['depot']
        depot = data['destination'][depot_idx]
        return depot

    def get_center_map(self, data):
        lat_locs = [des['geometry']['location']['lat'] for des in data['destination']]
        lng_locs = [des['geometry']['location']['lng'] for des in data['destination']]
        center_lat = np.mean(lat_locs)
        center_lng = np.mean(lng_locs)
        return center_lat, center_lng
    
    def get_covered_point(self, data):
        lat_locs = [des['geometry']['location']['lat'] for des in data['destination']]
        lng_locs = [des['geometry']['location']['lng'] for des in data['destination']]
        convered_lat = np.min(lat_locs) + ((np.max(lat_locs) - np.min(lat_locs))/ 2)
        convered_lng = np.min(lng_locs) + ((np.max(lng_locs) - np.min(lng_locs))/ 2)
        return convered_lat, convered_lng

    def parse_into_np_matrix(self, rows, size, dtype='distance'):
        np_matrix = np.zeros(size)
        for i, row in enumerate(rows):
            for j in range(len(row['elements'])):
                np_matrix[i][j] = row['elements'][j][dtype]['value']
        return np_matrix
    
    def get_matrix(self, google_matrix):
        if google_matrix['status'] == 'OK':
            size = (len(google_matrix['origin_addresses']), len(google_matrix['destination_addresses']))
            distance_matrix = self.parse_into_np_matrix(google_matrix['rows'], size)
            duration_matrix = self.parse_into_np_matrix(google_matrix['rows'], size, dtype='duration')
            return (distance_matrix, duration_matrix)
        return (None, None)

    def get_utility_matrices(self, origins, destinations, mode='driving'):
        logger.info('Gathering distance matrix and duration matrix')
        if 'datetime' in self.metadata.keys():
            self.time_marker = time.mktime(time.strptime(self.metadata['datetime'], "%d/%m/%Y %H:%M:%S"))
        else:
            self.time_marker = time.mktime(time.localtime(time.time()))
        
        # Get google matrix
        # 1. if static data is available, then search in it
        # 2. query directly by using Google MAP API
        if self.data is None or self.metric == 'duration':
            if len(origins) * len(destinations) < 100:
                gg_mat = self.gg_client.distance_matrix(origins=origins, destinations=destinations, mode=mode)
                return self.get_matrix(gg_mat)
            else:
                dist = None
                dura = None
                for i, origin in enumerate(origins):
                    gg_matrix = self.gg_client.distance_matrix(origins=origin, destinations=destinations, mode=mode)
                    distance_matrix, duration_matrix = self.get_matrix(gg_matrix)
                    if i == 0:
                        dist = distance_matrix
                        dura = duration_matrix
                    else:
                        dist = np.concatenate((dist, distance_matrix), axis=0)
                        dura = np.concatenate((dura, duration_matrix), axis=0)
                return dist, dura
        else:
            static_place_ids = {k['place_id']: i for i, k in self.static_data['geocode'].items()}
            place_ids = {i: k['place_id'] for i, k in enumerate(self.problem['destination'])}
            ids = np.array([int(static_place_ids[place_ids[i]]) for i in range(len(origins))])
            full_matrix = self.static_data['distance_matrix']
            matrix = np.zeros((len(ids), len(ids)))
            for i, idx_i in enumerate(ids):
                for j, idx_j in enumerate(ids):
                    matrix[i][j] = full_matrix[idx_i][idx_j]
            return matrix.astype(int).tolist(), None
    
    def get_direction_description(self, tour):
        if tour is not None:
            if self.static_data is None:
                descriptions = []
                for i in range(len(tour) - 1):
                    origin = self.get_location(self.problem['destination'][tour[i] - 1])
                    target = self.get_location(self.problem['destination'][tour[i + 1] - 1])
                    directions_result = self.gg_client.directions(origin, target, mode="driving",)
                    descriptions.append(directions_result)
                origin = self.get_location(self.problem['destination'][tour[-1] - 1])
                target = self.get_location(self.problem['destination'][tour[0] - 1])
                directions_result = self.gg_client.directions(origin, target, mode="driving",)
                descriptions.append(directions_result)
                return descriptions
            else:
                static_place_ids = {k['place_id']: i for i, k in self.static_data['geocode'].items()}
                place_ids = {i: k['place_id'] for i, k in enumerate(self.problem['destination'])}
                ids = np.array([int(static_place_ids[place_ids[i - 1]]) for i in tour])
                descriptions = []
                for i in range(len(tour) - 1):
                    origin_placeid = self.problem['destination'][tour[i] - 1]['place_id']
                    target_placeid = self.problem['destination'][tour[i + 1] - 1]['place_id']
                    origin_id = static_place_ids[origin_placeid]
                    target_id = static_place_ids[target_placeid]
                    directions_result = self.static_data['direction'][origin_id][target_id]
                    descriptions.append(directions_result)
                origin = self.problem['destination'][tour[-1] - 1]['place_id']
                target = self.problem['destination'][tour[0] - 1]['place_id']
                origin_id = static_place_ids[origin_placeid]
                target_id = static_place_ids[target_placeid]
                directions_result = self.static_data['direction'][origin_id][target_id]
                descriptions.append(directions_result)
                return descriptions
        return None
        
    
class TSPTWVis:
    def __init__(self, tsptw_data, layout=None, figsize=(1400, 800), rps=1):
        gmaps.configure(api_key=os.getenv('GOOGLE_API_KEY'))
        self.tsptw_data = tsptw_data
        self.figsize = figsize
        self.layout = layout
        self.rps = rps

    def draw_figure(self, save=True):
        if self.tsptw_data.optimal_tsp:
            if self.layout is None:
                layout = {
                    'width': "{}px".format(self.figsize[0]),
                    'height': "{}px".format(self.figsize[1]),}
            else:
                layout = {
                    'width': self.layout,
                    'height': self.layout}
            fig = gmaps.figure(layout=layout)
            locations = [self.tsptw_data.destinations[route - 1] for route in self.tsptw_data.optimal_tsp]
            labels = [str(i) if i != self.tsptw_data.problem['depot'] else 'Depot' for i in range(len(self.tsptw_data.optimal_tsp))]
            info_box_contents = []
            for idx in self.tsptw_data.optimal_tsp:
                address = self.tsptw_data.addresses[idx - 1]
                tw = self.tsptw_data.problem['time_window'][idx - 1]
                time_marker = self.tsptw_data.time_marker 
                opened_tw = time.strftime("%d/%m/%y %H:%M:%S", time.localtime(time_marker + tw[0]))
                closed_tw = time.strftime("%d/%m/%y %H:%M:%S", time.localtime(time_marker + tw[1]))
                content = """
                <div class="poi-info-window gm-style"> 
                <div jstcache="126" class="title full-width" jsan="7.title,7.full-width">{}</div>
                <div jstcache="127" jsinstance="1" class="address-line full-width" jsan="7.address-line,7.full-width">Open: {}</div>
                <div jstcache="127" jsinstance="1" class="address-line full-width" jsan="7.address-line,7.full-width">Close: {}</div>
                </div>""".format(address, opened_tw, closed_tw)
                info_box_contents.append(content)
            makers_layer = gmaps.marker_layer(locations, label=labels, info_box_content=info_box_contents)
            fig.add_layer(makers_layer)
            fig, route_layer_apis = self.add_multiple_directions_layer(fig, locations)
            if save is True:
                self.save_figure_to_html(fig)
            return fig, route_layer_apis
        else:
            print('Unable to find the solution')

    def add_multiple_directions_layer(self, fig, locations):
        locations += [locations[0]]
        list_waypoints = [DirectionDescriptor(desc).get_waypoints() for desc in self.tsptw_data.descriptions]
        route_layer_apis = {}
        for i in range(len(locations) - 1):
            route_layer = gmaps.directions_layer(locations[i], locations[i+1], waypoints=list_waypoints[i], show_markers=False)
            route_layer_apis[route_layer._model_id] = route_layer
            # Avoid sending many requests
            time.sleep(self.rps)
            fig.add_layer(route_layer)
        return fig, route_layer_apis

    def save_figure_to_html(self, fig, output_loc='./tsptw_vis.html'):
        # Cannot export DirectionLayerView
        embed_minimal_html(output_loc, views=[fig])
        
    def print_readable_description(self):
        routes = []
        route_desc = []
        stdout = ''
        for i, desc in enumerate(self.tsptw_data.descriptions):
            if i ==0 :
                stdout += 'Depot\n'
            else:
                stdout += '{}\n'.format(i)
            if i < len(self.tsptw_data.descriptions) - 1:
                routes.append('{} -> {}'.format(i, i + 1))
            else:
                routes.append('{} -> {}'.format(i, 0))
            dir_des = DirectionDescriptor(desc)
            stdout += '{}\n'.format(dir_des)
            route_desc.append(dir_des.get_html())
        stdout += '{}\n'.format('Complete')
        return stdout, routes, route_desc

class DirectionDescriptor:
    def __init__(self, description):
        self.description = description[0]
        
    def __str__(self):
        self.start = self.description['legs'][0]['start_address']
        self.end = self.description['legs'][0]['end_address']
        message = 'START: {}\n'.format(self.start)
        for step in self.description['legs'][0]['steps']:
            message += '--- {}\n'.format(step['html_instructions'])
        message += 'END: {}\n'.format(self.end)
        soup = BeautifulSoup(message)
        text = soup.get_text()
        return text
    
    def get_html(self):
        self.start = self.description['legs'][0]['start_address']
        self.end = self.description['legs'][0]['end_address']
        message = '<p><b>START: {}</b></p>'.format(self.start)
        for step in self.description['legs'][0]['steps']:
            soup = BeautifulSoup(step['html_instructions'])
            text = soup.get_text()
            message += '<p>   {}</p>'.format(text)
        message += '<p><b>END: {}</b></p>'.format(self.end)
        return message
    
    def get_waypoints(self,):
        waypoints = []
        for step in self.description['legs'][0]['steps']:
            lat = step['end_location']['lat']
            lng = step['end_location']['lng']
            waypoints.append((lat, lng))
        return waypoints if len(waypoints) < 23 else waypoints[:23]